# Course Recommender

## Import Libraries

In [1]:
# data cleaning and EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# text analysis
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string

## Bring in Clean Data

In [2]:
df = pd.read_csv('clean_data.csv')

In [3]:
df.head(1)

,Unnamed: 0,name,instructor,level,category,subcategory,no_of_students,rating,no_of_rating,about,syllabus,language,url
0,0,Meditation: A way to achieve your goals in you...,Duck-Joo Lee,Beginner Level,Arts and Humanities,Philosophy,78489,4.6,843,Do we truly think that we have lived for ourse...,['Self- reflection is the methodology of medit...,English,https://www.coursera.org/learn/self-reflection...


In [4]:
str(df['syllabus'])

'0      [\'Self- reflection is the methodology of medit...\n1      [\'This course is about the remarkable northern...\n2      [\'Join us this week to find out how the course...\n3      [\'The goal of the Module is to define individu...\n4      [\'This module explores sourcing of materials f...\n                             ...                        \n906    [\'Approaching voting problems with the securit...\n907    [\'Welcome to our course! This is a hands-on wo...\n908    [\'Welcome to the first week of the course! Thı...\n909    [\'In this first week, you will learn more abou...\n910    ["This short module includes an overview of th...\nName: syllabus, Length: 911, dtype: object'

## Create the Corpus

I will combine all categorical columns except language.

In [5]:
df["corpus"] = df["name"].map(str) + " " + df["instructor"].map(str) +" "+ df["level"].map(str)+ " "+df['category'].map(str)+" "+df['subcategory'].map(str) +" "+ df['about'].map(str) + df['syllabus'].map(str) 
df["corpus"][0]


'Meditation: A way to achieve your goals in your life Duck-Joo Lee Beginner Level Arts and Humanities Philosophy Do we truly think that we have lived for ourselves? Perhaps we have lived for money, love, fame, family and pride etc.? Therefore, we don’t seem to be satisfied even though we are full of those things. It is because that we don’t know ourselves.Eric Fromm talked about human nature as two modes of being: “To Have” and “To Be”.  If we are “Having” the nature of possessions, we are not satisfied, and feel empty and futile. Then, how can we be “Being” the nature of our inner-selves?\n\nSometimes, we happened to be aware of this “Being” nature and try to change ourselves, but fail. But, because of our daily routines, it’s easily forgotten. And more it is hard to escape from our unwanted minds controlling us.\n\nFrom now on, let us reflect on ourselves and look at our minds leading up to today! Don’t we achieve our goals after knowing ourselves? And let us find the “Being” nature 

In [6]:
df.head()

,Unnamed: 0,name,instructor,level,category,subcategory,no_of_students,rating,no_of_rating,about,syllabus,language,url,corpus
0,0,Meditation: A way to achieve your goals in you...,Duck-Joo Lee,Beginner Level,Arts and Humanities,Philosophy,78489,4.6,843,Do we truly think that we have lived for ourse...,['Self- reflection is the methodology of medit...,English,https://www.coursera.org/learn/self-reflection...,Meditation: A way to achieve your goals in you...
1,1,Introduction to the Arctic: Climate,"Paul Myers, Ph.D",Mixed Level,Physical Science and Engineering,Environmental Science and Sustainability,13439,4.6,420,"The University of Alberta, the University of T...",['This course is about the remarkable northern...,English,https://www.coursera.org/learn/arctic-introduc...,Introduction to the Arctic: Climate Paul Myers...
2,2,Data Science in Stratified Healthcare and Prec...,Dr Areti Manataki,Intermediate Level,Data Science,Data Analysis,16421,4.6,241,An increasing volume of data is becoming avail...,['Join us this week to find out how the course...,English,https://www.coursera.org/learn/datascimed,Data Science in Stratified Healthcare and Prec...
3,3,How to Get Skilled: Introduction to Individual...,Dr. Valeri Chukhlomin,Intermediate Level,Personal Development,Personal Development,50038,3.7,214,Do you want to gain a competitive edge on the ...,['The goal of the Module is to define individu...,English,https://www.coursera.org/learn/skills-management,How to Get Skilled: Introduction to Individual...
4,4,Circular Economy - Sustainable Materials Manag...,Philip Peck,Beginner Level,Physical Science and Engineering,Environmental Science and Sustainability,37682,4.8,1136,This course looks at where important materials...,['This module explores sourcing of materials f...,English,https://www.coursera.org/learn/circular-economy,Circular Economy - Sustainable Materials Manag...


## Data Cleaning/Wrangling

- Explain cleaning and wrangling procedures

In [ ]:
stop_words = set(stopwords.words('english'))
stop_words.update(['course','module','introduction','will','week','learn','video','part','lecture','video','using','system','one','new','use','first','s','work'])

wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
    clean_text = []
    clean_text2 = []
    text = re.sub("'", "",text)
    text=re.sub("(\\d|\\W)+"," ",text) 
    text = text.replace("nbsp", "")
    clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
    clean_text2 = [word for word in clean_text if black_txt(word)]
    return " ".join(clean_text2) 

In [ ]:
print(stop_words)
print(len(stop_words))

In [ ]:
df['corpus'] = df['corpus'].apply(lambda x: clean_txt(x))

In [ ]:
df.head()

## Using TFIDF

In [ ]:
#initializing tfidf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
def get_cos_similarity_scores(user_input,vectorizer,vectorized_corpus):
    tokens = [str(tok) for tok in word_tokenize(user_input)]
    user_tfidf = vectorizer.transform(tokens)
    cos_similarity = map(lambda x: cosine_similarity(user_tfidf, x),vectorized_corpus)
    output = list(cos_similarity)
    return output

In [ ]:
def get_user_vectorized(user_input,vectorizer,vectorized_corpus):
    tokens = [str(tok) for tok in word_tokenize(user_input)]
    user_tfidf = vectorizer.transform(tokens)
    return user_tfidf

In [ ]:
TFIDF = tfidf_vectorizer.fit_transform((df['corpus']))

def get_recommendation_TFIDF(user_input,df):
    output = get_cos_similarity_scores(user_input,vectorizer = tfidf_vectorizer,vectorized_corpus =  TFIDF)
    top = sorted(range(len(output)), key=lambda i: output[i][0], reverse=True)[:5]
    list_scores = [output[i][0][0] for i in top]
    
    recommendation = pd.DataFrame(columns = ['name',  'instructor','rating', 'score','index'])
    count = 0
    best_index = list()
    for i in top:
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'instructor'] = df['instructor'][i]
        recommendation.at[count, 'rating'] = df['rating'][i]
        recommendation.at[count, 'score'] =  list_scores[count]
        index = list(df. index)
        recommendation['index'][count] = i
        count += 1
        best_index.append(i) 
    return recommendation, best_index

In [ ]:
get_recommendation_TFIDF('sql',df)[0]

## Using CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()

In [ ]:
CV = count_vectorizer.fit_transform((df['corpus']))

def get_recommendation_CV(user_input,df):
    output = get_cos_similarity_scores(user_input,vectorizer = count_vectorizer, vectorized_corpus= CV)
    top = sorted(range(len(output)), key=lambda i: output[i][0], reverse=True)[:5]
    list_scores = [output[i][0][0] for i in top]
    
    recommendation = pd.DataFrame(columns = ['name',  'instructor','rating', 'score','index'])
    count = 0
    best_index = list()
    for i in top:
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'instructor'] = df['instructor'][i]
        recommendation.at[count, 'rating'] = df['rating'][i]
        recommendation.at[count, 'score'] =  list_scores[count]
        index = list(df. index)
        recommendation['index'][count] = i
        count += 1
        best_index.append(i) 
    return recommendation, best_index

In [ ]:
user_input = locate_corpus_by_name('machine learning',df)

In [ ]:
get_recommendation_CV(user_input,df)[0]

## Using KNN

In [ ]:
from sklearn.neighbors import NearestNeighbors

def get_NNS(user_input,n_neighbors,vectorizer, vectorized_corpus):   # vectorized_corpus could be TFIDF or CV
    KNN = NearestNeighbors(n_neighbors, p=2)
    KNN.fit(vectorized_corpus)
    user_tfidf = get_user_vectorized(user_input,vectorizer,vectorized_corpus)
    NNs = KNN.kneighbors(user_tfidf, return_distance=True) 
    return NNs

In [ ]:
def get_recommendation_KNN(user_input,df):
    top = get_NNS(user_input,6,vectorizer=tfidf_vectorizer, vectorized_corpus=TFIDF)[1][0][1:]
    index_score = get_NNS(user_input,6,vectorizer=tfidf_vectorizer, vectorized_corpus=TFIDF)[0][0][1:]
    recommendation = pd.DataFrame(columns = ['name',  'instructor','rating', 'score','index'])
    count = 0
    best_index = list()
    for i in top:
        recommendation.at[count, 'name'] = df['name'][i]
        recommendation.at[count, 'instructor'] = df['instructor'][i]
        recommendation.at[count, 'rating'] = df['rating'][i]
        recommendation.at[count, 'score'] =  index_score[count]
        index = list(df. index)
        recommendation['index'][count] = i
        count += 1
        best_index.append(i) 
    return recommendation, best_index
    


In [ ]:
get_recommendation_KNN('tableau',df)[0]

In [ ]:
def locate_corpus_by_name(user_input,df):
    user_input = user_input.lower()
    df['name']= df['name'].str.lower()
    new = df.loc[df.name == user_input, 'corpus'].values[0]
    return new

In [ ]:
locate_corpus_by_name('Networks: Friends, Money, and Bytes',df)

In [ ]:
def recommender():
    response = input( "\033[1m" + """
                        Welcome to the Coursera course recommender! (It contains only free courses.)
                        Are you new here?(yes/no): """).lower().replace(" ", '')
    while response == 'yes':
        query_sentence = input("\033[1m" + """
                        I am glad you prefer studying today!
                        What do you want to learn? Give me some keywords that are important for you: 
                        """).lower()
            
        best_index = get_recommendation_CV(query_sentence,df)[1]
        #print(best_index)
        if len(best_index)==0:
            response = input("\033[1m" + """
                        Sorry, couldnt find a good match. Do you want to try again? (yes/no)
                        """)
            if response == 'yes':
                print("\033[1m"+ """
                        This time please give more details.
                        """)
            if response == 'no':
                print("\033[1m"+ """
                        Please don't quit on studying. See you later!
                        """)
            
        elif len(best_index)!=0:
            print("\033[1m"+"""
                        We have three courses for you, choose according to your level: """)
            
            for i in list(best_index):
                course_name = df['name'].iloc[i]
                instructor = df['instructor'].iloc[i]
                level = df['level'].iloc[i]
                print("\033[1m"+ """ 
                            {}: 
                            {} 
                            by {} """.format(level,"\u0332".join(course_name),instructor))
                    
            response = 'no'
            return 
        
    
            
    while response == 'no':
        query_sentence = str(input("\033[1m" + """
                            How nice you are back! Which course have you completed?(Full course name please)
                                """).lower())
    
        query = locate_corpus_by_name(query_sentence,df)
        #print(query)
        best_index = get_recommendation_CV(query,df)[1]
    
        #print(best_index)
        if len(best_index)==0:
            response = input("\033[1m" + """
                        Sorry, couldnt find a good match. Do you want to try again? (yes/no)
                        """)
            if response == 'yes':
                print("\033[1m"+ """
                        This time please give more details.
                        """)
            if response == 'no':
                print("\033[1m"+ """
                            Please don't quit on studying. See you later!
                        """)
            
        elif len(best_index)!=0:
            print("\033[1m"+"""
                        We have three courses for you, choose according to your level: """)
            
            for i in list(best_index):
                course_name = df['name'].iloc[i]
                instructor = df['instructor'].iloc[i]
                level = df['level'].iloc[i]
                print("\033[1m"+ """ 
                            {}: 
                            {} 
                            by {} """.format(level,"\u0332".join(course_name.title()),instructor))
                    
            response = 'yes'
            return
        

In [ ]:
recommender()

In [ ]:

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [ ]:
bunch_text = " ".join(text for text in df['corpus'][110].split(' '))

In [ ]:
stopwords = set(list(STOPWORDS))
stopwords.update(['course','module','introduction','will','week','learn','video','part','lecture','video','using','system','one','new','use','first','s','work'])

wordcloud = WordCloud(stopwords=stopwords, background_color="white", colormap= "magma").generate(bunch_text)

In [ ]:
plt.figure(figsize=[11,11])
plt.imshow(wordcloud, interpolation="sinc")
plt.axis("off")
title = df['name'].iloc[110]
plt.title('Course Name: ' + title, fontsize=15)

plt.show()


In [ ]:
def get_wordcloud(text):
    bunch_text = " ".join(text for text in text.split(' '))
    wordcloud = WordCloud(stopwords=stopwords, background_color="white", colormap= "magma").generate(bunch_text)
    plt.figure(figsize=[11,11])
    plt.imshow(wordcloud, interpolation="sinc")
    plt.axis("off")
    title = df['name'].iloc[110]
    plt.title('Course Name: ' + title, fontsize=15)

    plt.show()
    return

In [ ]:
get_wordcloud(df['corpus'][110])

In [ ]:
df.to_csv('to_tableau.csv')